# Part 3: Practical Data Preparation
**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading
Load the dataset.

In [2]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    # Load the CSV file using pandas
    df = pd.read_csv(file_path)
    return df

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [3]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # 1. Extract the categorical column
    cat_column = df[[column_to_encode]]
    # 2. Apply OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_col = encoder.fit_transform(cat_column)
    # 3. Create new column names
    feature_names = encoder.get_feature_names_out()
    # 4. Replace the original categorical column with the encoded columns
    encoded_df = pd.DataFrame(encoded_col, columns=feature_names, index=df.index)
    df = df.drop('smoker_status', axis=1)
    df = pd.concat([df, encoded_df], axis=1)
    return df

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [4]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # 1. Encode categorical features using the encode_categorical_features function
    df_encoded = encode_categorical_features(df, column_to_encode='smoker_status')
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    X = df_encoded.drop(columns = 'disease_outcome')
    y = df_encoded['disease_outcome']
    # 3. Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # 4. Handle missing values
    X_train = X_train.fillna(method='ffill')
    X_test = X_test.fillna(method='ffill')
    # Placeholder return - replace with your implementation
    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [5]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    # Return
    return X_train_resampled, y_train_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [12]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # Initialize and train a LogisticRegression model
    model = LogisticRegression(max_iter = 1000)
    model.fit(X_train, y_train)
    return model

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # 1. Generate predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    # 3. Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    # 4. Return metrics in a dictionary
    metrics = {
         "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc,
        "confusion_matrix": cm
    }
    # Return
    return metrics

## 7. Save Results

Save the evaluation metrics to a text file.

```python
# 1. Create 'results' directory if it doesn't exist
# 2. Format metrics as strings
# 3. Write metrics to 'results/results_part3.txt'
```

## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.

In [14]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    improvements = {}
    
    for metric in part1_metrics:
        part1_val = part1_metrics[metric]
        part3_val = part3_metrics[metric]
        improvement = ((part3_val - part1_val) / abs(part1_val)) * 100
        improvements[metric] = improvement
    return improvements

## 8. Main Execution

Run the complete workflow.

In [15]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    X_train = X_train.select_dtypes(include=[np.number])
    X_test = X_test.select_dtypes(include=[np.number])
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    os.makedirs("results", exist_ok=True)
    with open("results/results_part3.txt", "w") as f:
        for key, value in metrics.items():
            if isinstance(value, float):
                f.write(f"{key}: {value:.4f}\n")
            else:
                f.write(f"{key}: {value}\n")
    
    # 8. Load Part 1 results for comparison
    try:
        part1_metrics = {}
        with open('results/results_part1.txt', 'r') as f:
            for line in f:
                if ':' in line:
                    key, value = line.strip().split(':', 1)
                    try:
                        part1_metrics[key.strip()] = float(value.strip())
                    except ValueError:
                            continue
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

/tmp/ipykernel_2602/1542472401.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_train = X_train.fillna(method='ffill')
/tmp/ipykernel_2602/1542472401.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method='ffill')


accuracy: 0.8527
precision: 0.3849
recall: 0.8531
f1: 0.5304
auc: 0.9247

Model Comparison (improvement percentages):
accuracy: -7.00%
precision: -41.82%
recall: 183.72%
f1: 28.28%
auc: 1.80%
